<a href="https://colab.research.google.com/github/zying0113/Quantitative_Investment_Analysis/blob/main/Investment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 安裝FinMind

In [ ]:
pip install FinMind

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 2.1 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.5.25-py3-none-any.whl size=24878 sha256=82210db22109abc98032069fdf993b855cd30f2f32596bae2aec33b72a4

In [ ]:
#基本套件
import pandas as pd
import numpy as np
import datetime
from copy import deepcopy
import re
from functools import reduce
from collections import defaultdict, OrderedDict
from tqdm import trange, tqdm


# 繪圖工具
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly import subplots


#FinMind
from FinMind.data import DataLoader

# 資料庫使用

In [ ]:
stock_no = '2727'
start, end ='2021-01-01', '2023-05-03'

d1 = DataLoader()
stock = d1.taiwan_stock_daily(stock_id=stock_no, start_date=start, end_date=end)
stock.rename(columns={'date':'mdate', 'open':'open_d', 
                        'max':'high_d', 'min':'low_d',
                        'close':'close_d', 'Trading_Volume':'volume', }, inplace = True)
stock.drop(columns=['stock_id','Trading_money','spread','Trading_turnover'], inplace = True)
stock['mdate'] = pd.to_datetime(stock['mdate']) # 轉換日期格式('文字'轉'時間')

# 大盤
d2 = DataLoader()
market = d2.taiwan_stock_daily(stock_id='TAIEX', start_date=start, end_date=end)
market.rename(columns={'date':'mdate', 'open':'open_m', 
                        'max':'high_m', 'min':'low_m',
                        'close':'close_m', 'Trading_Volume':'volume_m', }, inplace = True)
market.drop(columns=['stock_id','Trading_money','spread','Trading_turnover'], inplace = True)
market['mdate'] = pd.to_datetime(market['mdate'])

# 技術型指標

In [ ]:
def MACD(DF, a = 12, b =26, c =9):
    '''
    簡單移動平均線: rolling
    指數移動平均線: ewm
    '''
    df = DF.copy()
    df["EMA_Fast"] = df['open_d'].ewm(span = a, min_periods = a).mean()
    df["EMA_Slow"] = df['open_d'].ewm(span = b, min_periods = b).mean()
    df["DIF"] = df["EMA_Fast"] - df["EMA_Slow"]
    df["MACD"] = df['DIF'].ewm(span = c, min_periods = c).mean()
    
    return df

In [ ]:
def BollingerBand(DF, n=20):
    '''
    標準差: std
    '''
    df = DF.copy()
    df['SMA'] = df['close_d'].rolling(window = n).mean()
    df['BBu'] = df['SMA'] + 2*df['close_d'].rolling(window = n).std(ddof=0)
    df['BBd'] = df['SMA'] - 2*df['close_d'].rolling(window = n).std(ddof=0)
    df['BB_width'] = df['BBu'] - df['BBd']
    df.dropna(inplace = True)
    
    return df

RSI指標的計算流程如下:
1.   依據隔日價差，計算 n日內平均的漲跌幅
2.   計算相對強度(RS)：n日內漲幅平均值 / n日內跌幅平均值
3.   計算相對強弱指標(RSI)：相對強度(RS) / (1 + 相對強度(RS)) × 100

In [ ]:
def RSI(DF, n=14):
    '''
    n日漲幅平均值 = n日內上漲日總上漲幅度加總 ÷ n
    n日跌幅平均值 = n日內下跌日總下跌幅度加總 ÷ n
    '''
    df = DF.copy()
    df['daliy_change'] = df['close_d'] - df['close_d'].shift(1)
    df['dUp'] = np.where(df['daliy_change'] >= 0, df['daliy_change'], 0)
    df['dDown'] = np.where(df['daliy_change'] < 0, -df['daliy_change'], 0)
    avg_dUp = []
    avg_dDown = []
    dUp = df['dUp'].tolist()
    dDown = df['dDown'].tolist()
    
    for i in range(len(df)):
        if i < n:
            avg_dUp.append(0)
            avg_dDown.append(0)
        elif i == n:
            avg_dUp.append(df['dUp'].ewm(span = n).mean()[n])
            avg_dDown.append(df['dDown'].ewm(span = n).mean()[n])
        else:
            avg_dUp.append(((n-1)*avg_dUp[i-1] + dUp[i])/n)
            avg_dDown.append(((n-1)*avg_dDown[i-1] + dDown[i])/n)
    
    df['avg_dUp'] = np.array(avg_dUp)
    df['avg_dDown'] = np.array(avg_dDown)
    df['RS'] = df['avg_dUp']/df['avg_dDown']
    df['RSI'] = df['RS'].apply(lambda x: x/(1+x) * 100)
        
    return df

In [ ]:
def ATR(DF, n=14):
    '''
    昨日收盤:close.shift()
    '''
    df = DF.copy()
    df['H-L'] = abs(df['high_d'] - df['low_d'])
    df['H-PC'] = abs(df['high_d'] - df['close_d'].shift())
    df['L-PC'] = abs(df['low_d'] - df['close_d'].shift())
    df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis =1, skipna =False)
    df['ATR'] = df['TR'].ewm(span =n, min_periods=n).mean()
    
    return df

In [ ]:
def KD(DF, n = 14):
    '''
    function to calculate KD
    '''
    df = DF.copy()
    df['High_14D'] = df['high_d'].rolling(n).max()
    df['Low_14D'] = df['low_d'].rolling(n).min()
    df['RSV'] = (df['close_d'] - df['Low_14D']) / (df['High_14D'] - df['Low_14D']) * 100
    df = df.dropna()    
    df['K'] = np.zeros(len(df))
    df['D'] = np.zeros(len(df))
    
    for i in range(len(df)):
        if i == 0:
            df['K'][i] = 50
            df['D'][i] = 50
        else:
            df['K'][i] = df['K'][i-1]*(2/3) + df['RSV'][i]*(1/3)
            df['D'][i] = df['D'][i-1]*(2/3) + df['K'][i]*(1/3)
    
    return df

# 績效計算function

將交易策略紀錄、與大盤資訊整合在一起，並計算報酬率及其他資訊

In [ ]:
def performance_cal(data, trade_book_, market):
  data_ = data.copy()
  data_ = data_.merge(trade_book_, on = 'mdate', how = 'outer').set_index('mdate')
  data_ = data_.merge(market, on = 'mdate', how = 'inner').set_index('mdate')

  # fillna after merge
  data_['CashValue'].fillna(method = 'ffill', inplace=True)
  data_['CashValue'].fillna(cash, inplace = True)
  data_['TradeUnit'].fillna(0, inplace = True)
  data_['HoldingPosition'] = data_['TradeUnit'].cumsum()

  # Calc strategy value and return
  data_["StockValue"] = [data_['open_d'][i] * data_['HoldingPosition'][i] *1000 for i in range(len(data_.index))]
  data_['TotalValue'] = data_['CashValue'] + data_['StockValue']
  data_['DailyValueChange'] = np.log(data_['TotalValue']) - np.log(data_['TotalValue']).shift(1)
  data_['AccDailyReturn'] =  (data_['TotalValue']/cash - 1) *100

  # Calc BuyHold return
  data_['AccBHReturn'] = (data_['open_d']/data_['open_d'][0] -1) * 100

  # Calc market return
  data_['AccMarketReturn'] = (data_['close_m'] / data_['close_m'][0] - 1) *100

  return data_

def num_output(data_):
  # Calc numerical output
  overallreturn = round((data_['TotalValue'][-1] / cash - 1) *100, 4) # 總績效
  num_buy, num_sell = len([i for i in data_.BuyOrSell if i == "Buy"]), len([i for i in data_.BuyOrSell if i == "Sell"]) # 買入次數與賣出次數
  num_trade = num_buy #交易次數

  avg_hold_period, avg_return = [], []
  tmp_period, tmp_return = [], []
  for i in range(len(trade_book_['mdate'])):
    if trade_book_['BuyOrSell'][i] == 'Buy':
        tmp_period.append(trade_book_["mdate"][i])
        tmp_return.append(trade_book_['CashFlow'][i])
    else:
        sell_date = trade_book_["mdate"][i]
        sell_price = trade_book_['CashFlow'][i] / len(tmp_return)
        avg_hold_period += [sell_date - j for j in tmp_period]
        avg_return += [ abs(sell_price/j) -1  for j in tmp_return]
        tmp_period, tmp_return = [], []

  avg_hold_period_, avg_return_ = np.mean(avg_hold_period), round(np.mean(avg_return) * 100,4) #平均持有期間，平均報酬
  max_win, max_loss = round(max(avg_return)*100, 4) , round(min(avg_return)*100, 4) # 最大獲利報酬，最大損失報酬
  winning_rate = round(len([i for i in avg_return if i > 0]) / len(avg_return) *100, 4)#勝率
  min_cash = round(min(data_['CashValue']),4) #最小現金持有量

  print(' 總績效:', overallreturn, '%\n', 
        '交易次數:', num_trade, '次\n',
        '買入次數:', num_buy, '次\n',
        '賣出次數:', num_sell, '次\n',
        '平均交易報酬:', avg_return_, '%\n',
        '平均交易報酬:', avg_return_, '%\n',
        '平均持有期間:', avg_hold_period_, '\n',
        '勝率:', winning_rate, '%\n',
        '最大獲利交易報酬:', max_win, '%\n',
        '最大損失交易報酬:', max_loss, '%\n',
        '最低現金持有量:', min_cash )

# 回測應用
---

*   買入信號  
 *  MACD 上升
 *  K值 > 70 且 高於前一天K值

*   賣出信號
 *  當日低點 < 前日收盤 - 前日ATR



In [ ]:
data = deepcopy(stock)
data = data.set_index('mdate')

data["K"] = KD(data)["K"]
data["DIF"] = MACD(data)["DIF"]
data["MACD"] = MACD(data)["MACD"]
data["ATR"] = ATR(data)["ATR"]

<ipython-input-72-f4396aa3e0d3>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-72-f4396aa3e0d3>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-72-f4396aa3e0d3>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-72-f4396aa3e0d3>:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [ ]:
principal = 500000
cash = principal
position = 0
order_unit = 0
trade_book = pd.DataFrame()

for i in range(data.shape[0] -2):
    
    DIF = data['DIF'][i]
    MACD = data['MACD'][i]
    K = data['K'][i]
    K_y = data['K'][i-1]
    ATR = data['ATR'][i-1]
    low_d = data['low_d'][i]
    close_d = data['close_d'][i-1]
    n_time = data.index[i + 1]
    n_open = data['open_d'][i + 1]
    
    
    if position == 0: #進場條件
        if DIF> MACD and K> 70 and K > K_y and cash >= n_open*1000: 
            position += 1
            order_time = n_time
            order_price = n_open
            order_unit = 1
            friction_cost = (20 if order_price*1000*0.001425 < 20 else order_price*1000*0.001425)
            total_cost = -1 * order_price * 1000 - friction_cost
            cash += total_cost
            trade_book = trade_book.append(
                pd.Series(
                [stock_no, 'Buy', order_time, 0, total_cost, order_unit, position, cash
                ]), ignore_index = True)
            
    elif position > 0:
        if low_d < close_d - ATR: # 出場條件
            order_unit = position
            position = 0
            cover_time = n_time
            cover_price = n_open
            friction_cost = (20 if cover_price*order_unit*1000*0.001425 < 20 else cover_price*order_unit*1000*0.001425) + cover_price*order_unit*1000*0.003
            total_cost = cover_price*order_unit*1000-friction_cost
            cash += total_cost
            trade_book = trade_book.append(pd.Series([
                stock_no, 'Sell', 0, cover_time, total_cost, -1*order_unit, position, cash
            ]), ignore_index=True)
            
        elif DIF> MACD and K> 70 and K > K_y and cash >= n_open*1000: #加碼
            order_unit = 1
            order_time = n_time
            order_price = n_open
            position += 1
            friction_cost = (20 if order_price*1000*0.001425 < 20 else order_price*1000*0.001425) 
            total_cost = -1 * order_price * 1000 - friction_cost
            cash += total_cost
            trade_book = trade_book.append(
            pd.Series(
            [
                stock_no, 'Buy', order_time, 0, total_cost, order_unit, position, cash
            ]), ignore_index = True)
            
if position > 0: # 最後一天平倉
    order_unit = position
    position = 0
    cover_price = data['open_d'][-1]
    cover_time = data.index[-1]
    friction_cost = (20 if cover_price*order_unit*1000*0.001425 < 20 else cover_price*order_unit*1000*0.001425) + cover_price*order_unit*1000*0.003
    cash += cover_price*order_unit*1000-friction_cost
    trade_book = trade_book.append(
    pd.Series(
    [
        stock_no, 'Sell',0, cover_time, cover_price*order_unit*1000-friction_cost, -1*order_unit, position, cash
    ]), ignore_index = True)    
    
trade_book.columns = ['Coid', 'BuyOrSell', 'BuyTime', 'SellTime', 'CashFlow','TradeUnit', 'HoldingPosition', 'CashValue']
# trade_book['AccCashFlow'] = trade_book.CashFlow.cumsum()
# trade_ = Performance(trade, 'Stock')

<ipython-input-75-d0f0930aafe6>:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-75-d0f0930aafe6>:55: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-75-d0f0930aafe6>:55: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-75-d0f0930aafe6>:43: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-75-d0f0930aafe6>:29: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-75-d0f0930aafe6>:43: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

In [ ]:
trade_book

,Coid,BuyOrSell,BuyTime,SellTime,CashFlow,TradeUnit,HoldingPosition,CashValue
0,2727,Buy,2021-03-03 00:00:00,0,-159226.5750,1,1,340773.4250
1,2727,Buy,2021-03-04 00:00:00,0,-164233.7000,1,2,176539.7250
2,2727,Buy,2021-03-05 00:00:00,0,-166236.5500,1,3,10303.1750
3,2727,Sell,0,2021-04-13 00:00:00,552544.1250,-3,0,562847.3000
4,2727,Buy,2021-04-22 00:00:00,0,-206293.5500,1,1,356553.7500
...,...,...,...,...,...,...,...,...
67,2727,Sell,0,2023-03-13 00:00:00,259347.2875,-1,0,530010.7750
68,2727,Buy,2023-03-27 00:00:00,0,-308438.9000,1,1,221571.8750
69,2727,Sell,0,2023-03-28 00:00:00,295187.9875,-1,0,516759.8625
70,2727,Buy,2023-03-30 00:00:00,0,-320956.7125,1,1,195803.1500


In [ ]:
trade_book_ = trade_book.copy()
trade_book_['mdate'] = [trade_book.BuyTime[i] if trade_book.BuyTime[i] != 0 else trade_book.SellTime[i] for i in trade_book.index]
trade_book_ = trade_book_.loc[:, ['BuyOrSell', 'CashFlow', 'TradeUnit', 'HoldingPosition', 'CashValue' ,'mdate']]

In [ ]:
cash = principal
data_ = performance_cal(data, trade_book_, market)
num_output(data_)

 總績效: -0.1093 %
 交易次數: 48 次
 買入次數: 48 次
 賣出次數: 24 次
 平均交易報酬: -0.1195 %
 平均交易報酬: -0.1195 %
 平均持有期間: 12 days 04:00:00 
 勝率: 31.25 %
 最大獲利交易報酬: 28.4442 %
 最大損失交易報酬: -11.6304 %
 最低現金持有量: 5382.2625


In [ ]:
#累積報酬圖

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = data_.index, y = data_.AccDailyReturn, mode = 'lines', name = '交易策略'
))
fig.add_trace(go.Scatter(
    x = data_.index, y = data_.AccBHReturn, mode = 'lines', name = '買進持有'
))
fig.add_trace(go.Scatter(
    x = data_.index, y = data_.AccMarketReturn, mode = 'lines', name = '市場大盤'
))
fig.update_layout(
    title = stock_no + '累積報酬圖', yaxis_title = '累積報酬(%)', xaxis_title = '時間'
)
fig.show()

In [ ]:
# 買賣訊號繪製圖
Buy_index = [i-1 for i in range(len(data_['BuyOrSell'])) if data_['BuyOrSell'][i] == 'Buy']
Sell_index = [i-1 for i in range(len(data_['BuyOrSell'])) if data_['BuyOrSell'][i] == 'Sell']
Buy_point, Sell_point = data_.iloc[Buy_index, :], data_.iloc[Sell_index, :]
fig = go.Figure()
fig.add_trace(go.Scatter(x = data_.index,
                        y = data_.close_d,
                        mode = 'lines', 
                        name = '收盤價格'))

fig.add_trace(
    go.Scatter(
        x = Buy_point.index, y = Buy_point.close_d, mode = 'markers', 
        marker_symbol="triangle-up", marker_color="red", marker_size=12, name = '買入訊號'
    )
)
fig.add_trace(
    go.Scatter(
        x = Sell_point.index, y = Sell_point.close_d, mode = 'markers', 
        marker_symbol="triangle-up", marker_color="limegreen", marker_size=12, name = '賣出訊號'
    )
)

# [MACD](https://medium.com/tej-api-%E9%87%91%E8%9E%8D%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90/%E9%87%8F%E5%8C%96%E5%88%86%E6%9E%90-%E5%85%AB-macd%E6%8C%87%E6%A8%99%E5%9B%9E%E6%B8%AC%E5%AF%A6%E6%88%B0-8794ca221029)
---
當快線(DIF)由下而上穿越慢線(MACD)時，代表股價有上漲的動能存在；反之快線(DIF)向下跌破慢線(MACD)時，代表股價下跌的機率相對高。



In [ ]:
data = MACD(stock, a = 12, b =26, c =9)
data = data.set_index('mdate')

## 交易策略

In [ ]:
principal = 500000
cash = principal
position = 0
order_unit = 0
trade_book = pd.DataFrame()
for i in range(data.shape[0] -2):
    
    DIF_1 = data['DIF'][i+1]
    DIF_2 = data['DIF'][i+2]
    MACD_1 = data['MACD'][i+1]
    MACD_2 = data['MACD'][i+2]
    n_time = data.index[i + 1]
    n_open = data['open_d'][i + 1]
    
    
    if position == 0: #進場條件
        if DIF_1>MACD_1 and DIF_2>MACD_2 and cash >= n_open*1000: 
            position += 1
            order_time = n_time
            order_price = n_open
            order_unit = 1
            friction_cost = (20 if order_price*1000*0.001425 < 20 else order_price*1000*0.001425)
            total_cost = -1 * order_price * 1000 - friction_cost
            cash += total_cost
            trade_book = trade_book.append(
                pd.Series(
                [stock_no, 'Buy', order_time, 0, total_cost, order_unit, position, cash
                ]), ignore_index = True)
            
    elif position > 0:
        if DIF_1<MACD_1 and DIF_2<MACD_2: # 出場條件
            order_unit = position
            position = 0
            cover_time = n_time
            cover_price = n_open
            friction_cost = (20 if cover_price*order_unit*1000*0.001425 < 20 else cover_price*order_unit*1000*0.001425) + cover_price*order_unit*1000*0.003
            total_cost = cover_price*order_unit*1000-friction_cost
            cash += total_cost
            trade_book = trade_book.append(pd.Series([
                stock_no, 'Sell', 0, cover_time, total_cost, -1*order_unit, position, cash
            ]), ignore_index=True)
            
        elif DIF_1>MACD_1 and DIF_2>MACD_2 and cash >= n_open*1000: 
            order_unit = 1
            order_time = n_time
            order_price = n_open
            position += 1
            friction_cost = (20 if order_price*1000*0.001425 < 20 else order_price*1000*0.001425) 
            total_cost = -1 * order_price * 1000 - friction_cost
            cash += total_cost
            trade_book = trade_book.append(
            pd.Series(
            [
                stock_no, 'Buy', order_time, 0, total_cost, order_unit, position, cash
            ]), ignore_index = True)
            
if position > 0: # 最後一天平倉
    order_unit = position
    position = 0
    cover_price = data['open_d'][-1]
    cover_time = data.index[-1]
    friction_cost = (20 if cover_price*order_unit*1000*0.001425 < 20 else cover_price*order_unit*1000*0.001425) + cover_price*order_unit*1000*0.003
    cash += cover_price*order_unit*1000-friction_cost
    trade_book = trade_book.append(
    pd.Series(
    [
        stock_no, 'Sell',0, cover_time, cover_price*order_unit*1000-friction_cost, -1*order_unit, position, cash
    ]), ignore_index = True)    
    
trade_book.columns = ['Coid', 'BuyOrSell', 'BuyTime', 'SellTime', 'CashFlow','TradeUnit', 'HoldingPosition', 'CashValue']
# trade_book['AccCashFlow'] = trade_book.CashFlow.cumsum()
# trade_ = Performance(trade, 'Stock')

<ipython-input-51-8c2cdd8c4b5a>:25: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-51-8c2cdd8c4b5a>:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-51-8c2cdd8c4b5a>:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-51-8c2cdd8c4b5a>:39: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-51-8c2cdd8c4b5a>:25: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-51-8c2cdd8c4b5a>:39: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

## 交易紀錄

In [ ]:
trade_book

,Coid,BuyOrSell,BuyTime,SellTime,CashFlow,TradeUnit,HoldingPosition,CashValue
0,2727,Buy,2021-03-02 00:00:00,0,-1.632323e+05,1,1,3.367677e+05
1,2727,Buy,2021-03-03 00:00:00,0,-1.592266e+05,1,2,1.775411e+05
2,2727,Buy,2021-03-04 00:00:00,0,-1.642337e+05,1,3,1.330745e+04
3,2727,Sell,0,2021-03-23 00:00:00,5.391039e+05,-3,0,5.524113e+05
4,2727,Buy,2021-04-21 00:00:00,0,-2.002850e+05,1,1,3.521263e+05
...,...,...,...,...,...,...,...,...
83,2727,Sell,0,2023-03-10 00:00:00,1.077212e+06,-4,0,1.078588e+06
84,2727,Buy,2023-03-24 00:00:00,0,-3.224588e+05,1,1,7.561296e+05
85,2727,Buy,2023-03-27 00:00:00,0,-3.084389e+05,1,2,4.476907e+05
86,2727,Buy,2023-03-28 00:00:00,0,-2.969225e+05,1,3,1.507682e+05


In [ ]:
trade_book_ = trade_book.copy()
trade_book_['mdate'] = [trade_book.BuyTime[i] if trade_book.BuyTime[i] != 0 else trade_book.SellTime[i] for i in trade_book.index]
trade_book_ = trade_book_.loc[:, ['BuyOrSell', 'CashFlow', 'TradeUnit', 'HoldingPosition', 'CashValue' ,'mdate']]

## 績效計算

In [ ]:
cash = principal
data_ = performance_cal(data, trade_book_, market)
num_output(data_)

 總績效: 110.2532 %
 交易次數: 69 次
 買入次數: 69 次
 賣出次數: 19 次
 平均交易報酬: 5.3471 %
 平均交易報酬: 5.3471 %
 平均持有期間: 21 days 15:39:07.826086956 
 勝率: 49.2754 %
 最大獲利交易報酬: 59.1242 %
 最大損失交易報酬: -11.0668 %
 最低現金持有量: 1376.275


## 繪圖

In [ ]:
#累積報酬圖

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = data_.index, y = data_.AccDailyReturn, mode = 'lines', name = '交易策略'
))
fig.add_trace(go.Scatter(
    x = data_.index, y = data_.AccBHReturn, mode = 'lines', name = '買進持有'
))
fig.add_trace(go.Scatter(
    x = data_.index, y = data_.AccMarketReturn, mode = 'lines', name = '市場大盤'
))
fig.update_layout(
    title = stock_no + '累積報酬圖', yaxis_title = '累積報酬(%)', xaxis_title = '時間'
)
fig.show()

In [ ]:
# 買賣訊號繪製圖
Buy_index = [i-1 for i in range(len(data_['BuyOrSell'])) if data_['BuyOrSell'][i] == 'Buy']
Sell_index = [i-1 for i in range(len(data_['BuyOrSell'])) if data_['BuyOrSell'][i] == 'Sell']
Buy_point, Sell_point = data_.iloc[Buy_index, :], data_.iloc[Sell_index, :]
fig = go.Figure()
fig.add_trace(go.Scatter(x = data_.index,
                        y = data_.close_d,
                        mode = 'lines', 
                        name = '收盤價格'))

fig.add_trace(
    go.Scatter(
        x = Buy_point.index, y = Buy_point.close_d, mode = 'markers', 
        marker_symbol="triangle-up", marker_color="red", marker_size=12, name = '買入訊號'
    )
)
fig.add_trace(
    go.Scatter(
        x = Sell_point.index, y = Sell_point.close_d, mode = 'markers', 
        marker_symbol="triangle-up", marker_color="limegreen", marker_size=12, name = '賣出訊號'
    )
)

fig.add_vrect(
    x0="2021-11-02", x1="2021-12-20",
    fillcolor="lightBlue", opacity=0.5,
    layer="below", line_width=0,
)

fig.add_vrect(
    x0="2022-03-23", x1="2022-08-05",
    fillcolor="LightGreen", opacity=0.5,
    layer="below", line_width=0,
)

fig.update_layout(
    title = stock_no + '股價走勢圖', yaxis_title = '股票價格', xaxis_title = '時間',
)

fig.show()

# [布林通道](https://medium.com/tej-api-%E9%87%91%E8%9E%8D%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90/%E5%AF%A6%E6%88%B0%E6%87%89%E7%94%A8-%E5%B8%83%E6%9E%97%E9%80%9A%E9%81%93%E4%BA%A4%E6%98%93%E7%AD%96%E7%95%A5-737b65faac2f)


---



當收盤價觸碰到上界時，視為接下來可能會下跌的訊號，以隔日開盤價拋售持有部位。

當收盤價觸碰到下界時，視為接下來有可能谷底反彈的訊號，以隔日開盤價買入一單位。當滿足上述條件時，以及滿足本金充足、已持有部位與當日收盤價低於上次買入訊號收盤價時，則繼續加碼一單位。

In [ ]:
data = BollingerBand(stock)
data = data.set_index('mdate')

In [ ]:
fig = px.line(data,   
            x=data.index, 
            y=["close_d","BBu","BBd"], 
            color_discrete_sequence = px.colors.qualitative.Vivid
            )
fig.show()

## 交易策略

逆勢操作:\
若收盤價觸碰BBU20下界，以隔日開盤價買入；若收盤價觸碰BBU20上界，以隔日開盤價賣出。\
若已持有部位，上述條件滿足且本金充分時，則多買入一張

In [ ]:
principal = 500000
cash = principal
position = 0
order_unit = 0
trade_book = pd.DataFrame()

for i in range(data.shape[0] -2):
    
    cu_time = data.index[i]
    cu_close = data.loc[cu_time, 'close_d']
    cu_bbl, cu_bbu = data.loc[cu_time, 'BBd'], data.loc[cu_time, 'BBu']
    n_time = data.index[i + 1]
    n_open = data['open_d'][i + 1]
    
    
    if position == 0: #進場條件
        if cu_close <= cu_bbl and cash >= n_open*1000: 
            position += 1
            order_time = n_time
            order_price = n_open
            order_unit = 1
            friction_cost = (20 if order_price*1000*0.001425 < 20 else order_price*1000*0.001425)
            total_cost = -1 * order_price * 1000 - friction_cost
            cash += total_cost
            trade_book = trade_book.append(
                pd.Series(
                [stock_no, 'Buy', order_time, 0, total_cost, order_unit, position, cash
                ]), ignore_index = True)
            
    elif position > 0:
        if cu_close >= cu_bbu: # 出場條件
            order_unit = position
            position = 0
            cover_time = n_time
            cover_price = n_open
            friction_cost = (20 if cover_price*order_unit*1000*0.001425 < 20 else cover_price*order_unit*1000*0.001425) + cover_price*order_unit*1000*0.003
            total_cost = cover_price*order_unit*1000-friction_cost
            cash += total_cost
            trade_book = trade_book.append(pd.Series([
                stock_no, 'Sell', 0, cover_time, total_cost, -1*order_unit, position, cash
            ]), ignore_index=True)
            
        elif cu_close <= cu_bbl and cu_close <= order_price and cash >= n_open*1000: #加碼條件: 碰到下界，比過去買入價格貴
            order_unit = 1
            order_time = n_time
            order_price = n_open
            position += 1
            friction_cost = (20 if order_price*1000*0.001425 < 20 else order_price*1000*0.001425) 
            total_cost = -1 * order_price * 1000 - friction_cost
            cash += total_cost
            trade_book = trade_book.append(
            pd.Series(
            [
                stock_no, 'Buy', order_time, 0, total_cost, order_unit, position, cash
            ]), ignore_index = True)
            
if position > 0: # 最後一天平倉
    order_unit = position
    position = 0
    cover_price = data['open_d'][-1]
    cover_time = data.index[-1]
    friction_cost = (20 if cover_price*order_unit*1000*0.001425 < 20 else cover_price*order_unit*1000*0.001425) + cover_price*order_unit*1000*0.003
    cash += cover_price*order_unit*1000-friction_cost
    trade_book = trade_book.append(
    pd.Series(
    [
        stock_no, 'Sell',0, cover_time, cover_price*order_unit*1000-friction_cost, -1*order_unit, position, cash
    ]), ignore_index = True)    
    
trade_book.columns = ['Coid', 'BuyOrSell', 'BuyTime', 'SellTime', 'CashFlow','TradeUnit', 'HoldingPosition', 'CashValue']
# trade_book['AccCashFlow'] = trade_book.CashFlow.cumsum()
# trade_ = Performance(trade, 'Stock')

<ipython-input-66-046fc95e8bc7>:25: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-66-046fc95e8bc7>:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-66-046fc95e8bc7>:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-66-046fc95e8bc7>:39: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-66-046fc95e8bc7>:25: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-66-046fc95e8bc7>:51: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

## 交易紀錄

In [ ]:
trade_book

,Coid,BuyOrSell,BuyTime,SellTime,CashFlow,TradeUnit,HoldingPosition,CashValue
0,2727,Buy,2021-05-04 00:00:00,0,-174247.9500,1,1,325752.0500
1,2727,Buy,2021-05-05 00:00:00,0,-169240.8250,1,2,156511.2250
2,2727,Buy,2021-05-10 00:00:00,0,-154219.4500,1,3,2291.7750
3,2727,Sell,0,2021-07-02 00:00:00,483849.4500,-3,0,486141.2250
4,2727,Buy,2021-08-09 00:00:00,0,-147209.4750,1,1,338931.7500
5,2727,Buy,2021-08-17 00:00:00,0,-137195.2250,1,2,201736.5250
6,2727,Buy,2021-08-18 00:00:00,0,-134190.9500,1,3,67545.5750
7,2727,Sell,0,2021-09-29 00:00:00,467422.4625,-3,0,534968.0375
8,2727,Buy,2021-11-30 00:00:00,0,-136694.5125,1,1,398273.5250
9,2727,Buy,2021-12-03 00:00:00,0,-135192.3750,1,2,263081.1500


In [ ]:
trade_book_ = trade_book.copy()
trade_book_['mdate'] = [trade_book.BuyTime[i] if trade_book.BuyTime[i] != 0 else trade_book.SellTime[i] for i in trade_book.index]
trade_book_ = trade_book_.loc[:, ['BuyOrSell', 'CashFlow', 'TradeUnit', 'HoldingPosition', 'CashValue' ,'mdate']]

## 績效計算

In [ ]:
cash = principal
data_ = performance_cal(data, trade_book_, market)
num_output(data_)

 總績效: 23.3266 %
 交易次數: 19 次
 買入次數: 19 次
 賣出次數: 6 次
 平均交易報酬: 5.0814 %
 平均交易報酬: 5.0814 %
 平均持有期間: 47 days 05:03:09.473684210 
 勝率: 57.8947 %
 最大獲利交易報酬: 22.0297 %
 最大損失交易報酬: -7.4404 %
 最低現金持有量: 2291.775


In [ ]:
data_

,volume,open_d,high_d,low_d,close_d,SMA,BBu,BBd,BB_width,BuyOrSell,...,open_m,high_m,low_m,close_m,StockValue,TotalValue,DailyValueChange,AccDailyReturn,AccBHReturn,AccMarketReturn
mdate,,,,,,,,,,,,,,,,,,,,,
2021-01-29,4564395,125.5,127.5,113.0,114.0,126.175,143.671500,108.678500,34.992999,NaN,...,15544.36,15603.41,15138.31,15138.31,0.0,500000.00,NaN,0.00000,0.000000,0.000000
2021-02-01,1456026,113.0,117.0,109.5,117.0,125.825,143.756188,107.893812,35.862376,NaN,...,15176.56,15429.98,15089.96,15410.09,0.0,500000.00,0.0,0.00000,-9.960159,1.795313
2021-02-02,624055,115.0,116.5,114.5,115.0,125.125,143.590847,106.659153,36.931694,NaN,...,15546.69,15838.15,15546.69,15760.05,0.0,500000.00,0.0,0.00000,-8.366534,4.107063
2021-02-03,1269563,116.5,120.5,116.0,116.5,124.425,143.083175,105.766825,37.316350,NaN,...,15828.64,15896.40,15741.16,15771.32,0.0,500000.00,0.0,0.00000,-7.171315,4.181510
2021-02-04,420106,116.0,118.5,115.5,117.5,123.650,142.105622,105.194378,36.911245,NaN,...,15697.75,15801.37,15606.74,15706.22,0.0,500000.00,0.0,0.00000,-7.569721,3.751476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-26,6436171,313.0,314.5,298.0,300.0,306.375,333.830191,278.919809,54.910382,NaN,...,15352.03,15398.40,15284.46,15374.63,0.0,616632.95,0.0,23.32659,149.402390,1.561073
2023-04-27,4244617,299.0,311.0,295.5,309.0,307.075,334.044010,280.105990,53.938020,NaN,...,15374.39,15455.88,15306.69,15411.49,0.0,616632.95,0.0,23.32659,138.247012,1.804561
2023-04-28,5101011,314.0,320.0,309.0,317.0,307.875,335.024355,280.725645,54.298711,NaN,...,15477.34,15580.63,15477.34,15579.18,0.0,616632.95,0.0,23.32659,150.199203,2.912280


## 繪圖

In [ ]:
#累積報酬圖

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = data_.index, y = data_.AccDailyReturn, mode = 'lines', name = '交易策略'
))
fig.add_trace(go.Scatter(
    x = data_.index, y = data_.AccBHReturn, mode = 'lines', name = '買進持有'
))
fig.add_trace(go.Scatter(
    x = data_.index, y = data_.AccMarketReturn, mode = 'lines', name = '市場大盤'
))
fig.update_layout(
    title = stock_no + '累積報酬圖', yaxis_title = '累積報酬(%)', xaxis_title = '時間'
)
fig.show()

In [ ]:
# 買賣訊號繪製圖
Buy_index = [i-1 for i in range(len(data_['BuyOrSell'])) if data_['BuyOrSell'][i] == 'Buy']
Sell_index = [i-1 for i in range(len(data_['BuyOrSell'])) if data_['BuyOrSell'][i] == 'Sell']
Buy_point, Sell_point = data_.iloc[Buy_index, :], data_.iloc[Sell_index, :]
fig = go.Figure()
fig.add_trace(go.Scatter(x = data_.index,
                        y = data_.close_d,
                        mode = 'lines', 
                        name = '收盤價格'))
fig.add_trace(
    go.Scatter(
        x = data_.index, y = data_.BBu, mode = 'lines',name = '上界'
    )
)
fig.add_trace(
    go.Scatter(
        x = data_.index, y = data_.BBd, mode = 'lines',name = '下界'
    )
)

fig.add_trace(
    go.Scatter(
        x = Buy_point.index, y = Buy_point.close_d, mode = 'markers',
        marker_symbol="triangle-up", marker_color="red", marker_size=12, name = '買入訊號'
    )
)
fig.add_trace(
    go.Scatter(
        x = Sell_point.index, y = Sell_point.close_d, mode = 'markers',
        marker_symbol="triangle-up", marker_color="limegreen", marker_size=12, name = '賣出訊號'
    )
)

fig.add_vrect(
    x0="2021-11-02", x1="2021-12-20",
    fillcolor="lightBlue", opacity=0.5,
    layer="below", line_width=0,
)

fig.add_vrect(
    x0="2022-03-23", x1="2022-08-05",
    fillcolor="LightGreen", opacity=0.5,
    layer="below", line_width=0,
)

fig.update_layout(
    title = stock_no + '股價走勢圖', yaxis_title = '股票價格', xaxis_title = '時間',
)

fig.show()

#[RSI指標策略](https://medium.com/tej-api-%E9%87%91%E8%9E%8D%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90/%E9%87%8F%E5%8C%96%E5%88%86%E6%9E%90-%E5%8D%81%E4%B8%89-rsi%E6%8C%87%E6%A8%99%E5%9B%9E%E6%B8%AC%E5%AF%A6%E6%88%B0-1d915873f1f7)
---
RSI指標的判斷依據如下：
1.   RSI < 30為賣超情形，RSI > 70為買超情形，RSI在50之間波動為買賣方力量持平，上述(30,70)組合也有較保守的(20,80)判斷依據。
2.   RSI指標背離，也就是RSI走勢與股價走勢不一致，代表市場將出現反轉，是買賣訊號的一個依據。
3.   黃金交叉，短天期 RSI向上突破長天期 RSI，代表市場即將進入多頭。
4.   死亡交叉，短天期 RSI向下突破長天期 RSI，代表市場即將進入空頭。





In [ ]:
data_14 = RSI(stock,14)
data_7 = RSI(stock,7)
data = data_14.merge(data_7, on = ['mdate','open_d','high_d','low_d','close_d','volume','daliy_change','dUp','dDown'])

In [ ]:
data.rename(columns={'avg_dUp_x':'avg_dUp14', 'avg_dDown_x':'avg_dDown14', 
                        'RS_x':'RS14', 'RSI_x':'RSI14',
                        'avg_dUp_y':'avg_dUp7', 'avg_dDown_y':'avg_dDown7', 
                        'RS_y':'RS7', 'RSI_y':'RSI7',}, inplace = True)
data

,mdate,volume,open_d,high_d,low_d,close_d,daliy_change,dUp,dDown,avg_dUp14,avg_dDown14,RS14,RSI14,avg_dUp7,avg_dDown7,RS7,RSI7
0,2021-01-04,1237895,127.0,127.0,123.0,124.0,NaN,0.0,0.0,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN
1,2021-01-05,1226259,124.5,130.0,124.5,129.0,5.0,5.0,0.0,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN
2,2021-01-06,1281983,129.0,131.5,127.0,130.5,1.5,1.5,0.0,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN
3,2021-01-07,1878853,131.5,136.0,129.0,133.0,2.5,2.5,0.0,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN
4,2021-01-08,3441669,136.5,141.5,133.5,140.5,7.5,7.5,0.0,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,2023-04-26,6436171,313.0,314.5,298.0,300.0,-15.5,0.0,15.5,5.284220,5.169511,1.022189,50.548649,5.303476,6.702129,0.791312,44.175001
559,2023-04-27,4244617,299.0,311.0,295.5,309.0,9.0,9.0,0.0,5.549633,4.800261,1.156111,53.620193,5.831551,5.744682,1.015122,50.375204
560,2023-04-28,5101011,314.0,320.0,309.0,317.0,8.0,8.0,0.0,5.724659,4.457385,1.284309,56.223084,6.141330,4.924013,1.247220,55.500581
561,2023-05-02,1802118,320.0,322.5,314.5,317.0,0.0,0.0,0.0,5.315755,4.139000,1.284309,56.223084,5.263997,4.220583,1.247220,55.500581


In [ ]:
data = data.reset_index()

fig = go.Figure()


fig.add_trace(go.Scatter(x = data['mdate'], y = data['RSI14'], name = "14-RSI")) 
fig.add_trace(go.Scatter(x = data['mdate'], y = data['RSI7'], name = "7-RSI"))
fig.add_trace(go.Scatter(x = data['mdate'], y = np.full(shape=500, fill_value = 70), line = dict(color='grey', width=3, dash='dash'), name = '70RSI'))                       
fig.add_trace(go.Scatter(x = data['mdate'], y = np.full(shape=500, fill_value = 30), line = dict(color='grey', width=3, dash='dash'), name = '30RSI'))                       

graph_gold = go.Scatter(x=data['mdate'], y= np.where((data['RSI7'].shift() < data['RSI14'].shift()) & (data['RSI7'] >= data['RSI14']) & (data['RSI14'] < 30),
                                                        data['RSI14'], np.nan), 
                            mode='markers', marker_symbol="x", marker_color="gold", marker_size=15, name='黃金交叉')

graph_dead = go.Scatter(x=data["mdate"], y= np.where((data['RSI7'].shift() > data['RSI14'].shift()) & (data['RSI7'] <= data['RSI14']) & (data['RSI14'] > 70), 
                                                        data['RSI14'], np.nan), 
                            mode='markers', marker_symbol="x", marker_color="black", marker_size=10, name='死亡交叉')

fig.add_trace(graph_gold)
fig.add_trace(graph_dead)

fig.update_layout(title='RSI走勢圖',
                 xaxis_title="日期",
                 yaxis_title="RSI")

fig.show()

## 交易策略

In [ ]:
data = data.set_index('mdate')
# 交易訊號只發生在買賣超階段與黃金交叉或死亡交叉同時發生的情境下

principal = 500000
cash = principal
position = 0
trade = 0
order_unit = 0
trade_book = pd.DataFrame()

for i in range(data.shape[0] -2):
    
    RSI_14 = data['RSI14'][i]
    RSI_14_y = data['RSI14'][i-1]
    RSI_7 = data['RSI7'][i]
    RSI_7_y = data['RSI7'][i-1]
    n_time = data.index[i + 1]
    n_open = data['open_d'][i + 1]
    
    
    if position == 0: #進場條件
        if RSI_14<=30 and RSI_14_y>RSI_7_y and RSI_14<=RSI_7 and trade==0 and cash >= n_open*1000: 
            position += 1
            order_time = n_time
            order_price = n_open
            order_unit = 1
            friction_cost = (20 if order_price*1000*0.001425 < 20 else order_price*1000*0.001425)
            total_cost = -1 * order_price * 1000 - friction_cost
            cash += total_cost
            trade_book = trade_book.append(
                pd.Series(
                [stock_no, 'Buy', order_time, 0, total_cost, order_unit, position, cash
                ]), ignore_index = True)
            
    elif position > 0:
        if RSI_14>=70 and RSI_14_y<RSI_7_y and RSI_14>=RSI_7 and trade==1: # 出場條件
            order_unit = position
            position = 0
            cover_time = n_time
            cover_price = n_open
            friction_cost = (20 if cover_price*order_unit*1000*0.001425 < 20 else cover_price*order_unit*1000*0.001425) + cover_price*order_unit*1000*0.003
            total_cost = cover_price*order_unit*1000-friction_cost
            cash += total_cost
            trade_book = trade_book.append(pd.Series([
                stock_no, 'Sell', 0, cover_time, total_cost, -1*order_unit, position, cash
            ]), ignore_index=True)
            
        elif RSI_14<=30 and RSI_14_y>RSI_7_y and RSI_14<=RSI_7 and trade==0 and cash >= n_open*1000: 
            order_unit = 1
            order_time = n_time
            order_price = n_open
            position += 1
            friction_cost = (20 if order_price*1000*0.001425 < 20 else order_price*1000*0.001425) 
            total_cost = -1 * order_price * 1000 - friction_cost
            cash += total_cost
            trade_book = trade_book.append(
            pd.Series(
            [
                stock_no, 'Buy', order_time, 0, total_cost, order_unit, position, cash
            ]), ignore_index = True)
            
if position > 0: # 最後一天平倉
    order_unit = position
    position = 0
    cover_price = data['open_d'][-1]
    cover_time = data.index[-1]
    friction_cost = (20 if cover_price*order_unit*1000*0.001425 < 20 else cover_price*order_unit*1000*0.001425) + cover_price*order_unit*1000*0.003
    cash += cover_price*order_unit*1000-friction_cost
    trade_book = trade_book.append(
    pd.Series(
    [
        stock_no, 'Sell',0, cover_time, cover_price*order_unit*1000-friction_cost, -1*order_unit, position, cash
    ]), ignore_index = True)    
    
trade_book.columns = ['Coid', 'BuyOrSell', 'BuyTime', 'SellTime', 'CashFlow','TradeUnit', 'HoldingPosition', 'CashValue']
# trade_book['AccCashFlow'] = trade_book.CashFlow.cumsum()
# trade_ = Performance(trade, 'Stock')

<ipython-input-41-5a474dc1876e>:30: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-41-5a474dc1876e>:69: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



## 交易紀錄

In [ ]:
trade_book

,Coid,BuyOrSell,BuyTime,SellTime,CashFlow,TradeUnit,HoldingPosition,CashValue
0,2727,Buy,2021-01-28 00:00:00,0,-116666.0125,1,1,383333.9875
1,2727,Sell,0,2023-05-03 00:00:00,315597.2750,-1,0,698931.2625


In [ ]:
trade_book_ = trade_book.copy()
trade_book_['mdate'] = [trade_book.BuyTime[i] if trade_book.BuyTime[i] != 0 else trade_book.SellTime[i] for i in trade_book.index]
trade_book_ = trade_book_.loc[:, ['BuyOrSell', 'CashFlow', 'TradeUnit', 'HoldingPosition', 'CashValue' ,'mdate']]

## 績效試算

In [ ]:
cash = principal
data_ = performance_cal(data, trade_book_, market)
num_output(data_)

總績效: 39.7863 %
交易次數: 1 次
買入次數: 1 次
賣出次數: 1 次
平均交易報酬: 170.5135 %
平均持有期間: 825 days 00:00:00
勝率: 100.0 %
最大獲利交易報酬: 170.5135 %
最大損失交易報酬: 170.5135 %
最低現金持有量: 383333.9875


In [ ]:
data_

,index,volume,open_d,high_d,low_d,close_d,daliy_change,dUp,dDown,avg_dUp14,...,open_m,high_m,low_m,close_m,StockValue,TotalValue,DailyValueChange,AccDailyReturn,AccBHReturn,AccMarketReturn
mdate,,,,,,,,,,,,,,,,,,,,,
2021-01-04,0,1237895,127.0,127.0,123.0,124.0,NaN,0.0,0.0,0.000000,...,14720.25,14937.13,14720.25,14902.03,0.0,500000.0000,NaN,0.000000,0.000000,0.000000
2021-01-05,1,1226259,124.5,130.0,124.5,129.0,5.0,5.0,0.0,0.000000,...,14913.64,15000.03,14861.99,15000.03,0.0,500000.0000,0.000000,0.000000,-1.968504,0.657629
2021-01-06,2,1281983,129.0,131.5,127.0,130.5,1.5,1.5,0.0,0.000000,...,15145.85,15197.68,14837.00,14983.13,0.0,500000.0000,0.000000,0.000000,1.574803,0.544221
2021-01-07,3,1878853,131.5,136.0,129.0,133.0,2.5,2.5,0.0,0.000000,...,15059.52,15270.40,15049.86,15214.00,0.0,500000.0000,0.000000,0.000000,3.543307,2.093473
2021-01-08,4,3441669,136.5,141.5,133.5,140.5,7.5,7.5,0.0,0.000000,...,15365.13,15463.95,15275.38,15463.95,0.0,500000.0000,0.000000,0.000000,7.480315,3.770761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-26,558,6436171,313.0,314.5,298.0,300.0,-15.5,0.0,15.5,5.284220,...,15352.03,15398.40,15284.46,15374.63,313000.0,696333.9875,-0.008580,39.266798,146.456693,3.171380
2023-04-27,559,4244617,299.0,311.0,295.5,309.0,9.0,9.0,0.0,5.549633,...,15374.39,15455.88,15306.69,15411.49,299000.0,682333.9875,-0.020310,36.466798,135.433071,3.418729
2023-04-28,560,5101011,314.0,320.0,309.0,317.0,8.0,8.0,0.0,5.724659,...,15477.34,15580.63,15477.34,15579.18,314000.0,697333.9875,0.021745,39.466798,147.244094,4.544012


## 繪圖

In [ ]:
#累積報酬圖

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = data_.index, y = data_.AccDailyReturn, mode = 'lines', name = '交易策略'
))
fig.add_trace(go.Scatter(
    x = data_.index, y = data_.AccBHReturn, mode = 'lines', 
    marker_symbol="triangle-up", marker_color="red", marker_size=12, name = '買進持有'
))
fig.add_trace(go.Scatter(
    x = data_.index, y = data_.AccMarketReturn, mode = 'lines',
    marker_symbol="triangle-up", marker_color="limegreen", marker_size=12, name = '市場大盤'
))
fig.update_layout(
    title = stock_no + '累積報酬圖', yaxis_title = '累積報酬(%)', xaxis_title = '時間'
)
fig.show()

In [ ]:
# 買賣訊號繪製圖
Buy_index = [i-1 for i in range(len(data_['BuyOrSell'])) if data_['BuyOrSell'][i] == 'Buy']
Sell_index = [i-1 for i in range(len(data_['BuyOrSell'])) if data_['BuyOrSell'][i] == 'Sell']
Buy_point, Sell_point = data_.iloc[Buy_index, :], data_.iloc[Sell_index, :]
fig = go.Figure()
fig.add_trace(go.Scatter(x = data_.index,
                        y = data_.close_d,
                        mode = 'lines', 
                        name = '收盤價格'))

fig.add_trace(
    go.Scatter(
        x = Buy_point.index, y = Buy_point.close_d, mode = 'markers', name = '買入訊號'
    )
)
fig.add_trace(
    go.Scatter(
        x = Sell_point.index, y = Sell_point.close_d, mode = 'markers', name = '賣出訊號'
    )
)

fig.add_vrect(
    x0="2021-11-02", x1="2021-12-20",
    fillcolor="lightBlue", opacity=0.5,
    layer="below", line_width=0,
)

fig.add_vrect(
    x0="2022-03-23", x1="2022-08-05",
    fillcolor="LightGreen", opacity=0.5,
    layer="below", line_width=0,
)

fig.update_layout(
    title = stock_no + '股價走勢圖', yaxis_title = '股票價格', xaxis_title = '時間',
)

fig.show()